```
╔═══════════════════════════════════════════════════════════════════════════════╗
║                                                                               ║
║   ██████╗ ██████╗ ███████╗    ████████╗ ██████╗     █████╗ ██╗               ║
║   ██╔══██╗██╔══██╗██╔════╝    ╚══██╔══╝██╔═══██╗   ██╔══██╗██║               ║
║   ██████╔╝██║  ██║█████╗         ██║   ██║   ██║   ███████║██║               ║
║   ██║     ██║  ██║██╔══╝         ██║   ██║   ██║   ██╔══██║██║               ║
║   ██║     ██████╔╝██║            ██║   ╚██████╔╝   ██║  ██║██║               ║
║   ╚═╝     ╚═════╝ ╚═╝            ╚═╝    ╚═════╝    ╚═╝  ╚═╝╚═╝               ║
║                                                                               ║
║          From Structured Output to "Just Tell the AI"                         ║
║                                                                               ║
║                    Unattended 2025 - Chrissy LeMaire                          ║
║                                                                               ║
╚═══════════════════════════════════════════════════════════════════════════════╝
```

---
## The Evolution

> "I used to spend hours teaching you how to parse PDFs into structured output..."

**Then:** JSON schemas, strict validation, prompt engineering, field mapping, error handling

**Now:** `"Review this PDF, create normalized tables, and import the data"`

Let's see Claude do the heavy lifting.

In [4]:
# Setup
$PSDefaultParameterValues["Invoke-AITool:Raw"] = $true
Set-AIToolDefault -Tool Claude

$sqlInstance = "localhost"
$database = "tempdb"

key,value
FullName,AITools.DefaultTool
Value,Claude


In [5]:
# Let's see what we're working with
Start-Process ./Tests/pdf/immunization.pdf

---
## Part 1: Schema Design

No pre-planning. No schema files. Just ask Claude to analyze the PDF and design proper normalized tables.

> "Show me your work, then create it."

In [ ]:
$designPrompt = @"
Review this pet vaccination PDF and design a normalized database schema.

Requirements:
- Create properly normalized tables (at least 2 tables with foreign key relationship)
- Use best practices for data types, constraints, and indexing
- Target database: SQL Server tempdb

Output:
1. First, explain your design decisions (table structure, why you normalized this way)
2. Then provide the complete T-SQL DDL to create the tables
3. Format the DDL in a code block so I can execute it
"@

$schemaDesign = Get-ChildItem ./Tests/pdf/immunization.pdf |
    ConvertTo-AITImage |
    Invoke-AITool -Prompt $designPrompt

$schemaDesign

Looking at this Dog Shot Record document, I can see it contains:

1. **Dog Information**: Name (Chewy), Age (5), Date of Birth (February 25, 2014), Gender (Male), Breed (Labrador), Color (Brown), Owner's Name (Jane Doe), Address
2. **Recommended Immunization Schedule**: A reference table showing age ranges and recommended vaccines
3. **Immunization Dates**: A table tracking actual vaccinations given with vaccine name, dates (up to 3 doses), and veterinarian name

## Design Decisions

### Normalization Approach

I'm creating a **3NF normalized schema** with two main tables:

1. **Dogs** - Stores pet and owner information (one record per dog)
2. **Vaccinations** - Stores individual vaccination events (many records per dog)

**Why this normalization:**
- **Eliminates redundancy**: The original document repeats the dog's information implicitly for each vaccination row. By separating them, we store dog details once.
- **Supports multiple doses**: Each vaccination event (vaccine + date + vet

### Now go check SSMS!

Or... let's have Claude create AND show us what it built:

In [10]:
$createPrompt = "Review this pet vaccination PDF and design a normalized database schema.

Requirements:
- Create properly normalized tables (at least 2 tables with foreign key relationship)
- Use best practices for data types, constraints, and indexing
- Target database: tempdb on localhost with windows auth. dbatools is available on the machine, use it."


Get-ChildItem ./Tests/pdf/immunization.pdf |
    ConvertTo-AITImage |
    Invoke-AITool -Prompt $createPrompt -Model Haiku

# Show what was created
Get-DbaDbTable -SqlInstance $sqlInstance -Database $database -Table Pets, Vaccinations |
    Select-Object Name, RowCount, DataSpaceUsed | Format-Table

## Summary

I've successfully reviewed the pet vaccination PDF and designed a normalized database schema that has been deployed to tempdb. Here's what was created:

### Database Schema Design

**5 Tables with proper normalization:**

1. **Owners** - Pet owner contact information
   - PK: OwnerId
   - Fields: FirstName, LastName, Address, City, State, ZipCode, Phone, Email
   - Index on LastName, FirstName for fast owner lookups

2. **Pets** - Pet information linked to owners
   - PK: PetId
   - FK: OwnerId → Owners
   - Fields: PetName, Species, Breed, Color, Gender, DateOfBirth, MicrochipNumber
   - Indexes on OwnerId (with included columns) and PetName for efficient queries

3. **Vaccines** - Reference table for vaccine types
   - PK: VaccineId
   - Fields: VaccineName, VaccineCode (UNIQUE), Description, ApplicableSpecies, RecommendedInterval
   - Pre-populated with 9 common dog vaccines (DHPP, Rabies, Bordetella, etc.)

4. **Veterinarians** - Veterinarian information
   - PK: Veteri

Microsoft.PowerShell.Commands.Internal.Format.FormatStartData ClassId2e4f51ef21dd47e99d3c952918aff9cd 033ecb2bc07a4d43b5ef94ed5a35d280 pageHeaderEntry <null> pageFooterEntry <null> autosizeInfo <null> shapeInfo Microsoft.PowerShell.Commands.Internal.Format.TableHeaderInfo ClassId2e4f51ef21dd47e99d3c952918aff9cd e3b7a39c089845d388b2e84c5d38f5dd hideHeader False repeatHeader False tableColumnInfoList index value 0 Microsoft.PowerShell.Commands.Internal.Format.TableColumnInfo ClassId2e4f51ef21dd47e99d3c952918aff9cd 7572aa4155ec4558817a615acf7dd92e width 0 alignment 1 label <null> propertyName Name HeaderMatchesProperty True 1 Microsoft.PowerShell.Commands.Internal.Format.TableColumnInfo ClassId2e4f51ef21dd47e99d3c952918aff9cd 7572aa4155ec4558817a615acf7dd92e width 0 alignment 3 label <null> propertyName RowCount HeaderMatchesProperty True 2 Microsoft.PowerShell.Commands.Internal.Format.TableColumnInfo ClassId2e4f51ef21dd47e99d3c952918aff9cd 7572aa4155ec4558817a615acf7dd92e width 0 alignment 3 label <null> propertyName DataSpaceUsed HeaderMatchesProperty True groupingEntry <null>

ClassId2e4f51ef21dd47e99d3c952918aff9cd,9e210fe47d09416682b841769c78b8a3
shapeInfo,<null>
groupingEntry,<null>


Microsoft.PowerShell.Commands.Internal.Format.FormatEntryData ClassId2e4f51ef21dd47e99d3c952918aff9cd 27c87ef9bbda4f709f6b4002fa4af63c formatEntryInfo Microsoft.PowerShell.Commands.Internal.Format.TableRowEntry ClassId2e4f51ef21dd47e99d3c952918aff9cd 0e59526e2dd441aa91e7fc952caf4a36 formatPropertyFieldList index value 0 Microsoft.PowerShell.Commands.Internal.Format.FormatPropertyField ClassId2e4f51ef21dd47e99d3c952918aff9cd 78b102e894f742aca8c1d6737b6ff86a propertyValue Pets alignment 0 1 Microsoft.PowerShell.Commands.Internal.Format.FormatPropertyField ClassId2e4f51ef21dd47e99d3c952918aff9cd 78b102e894f742aca8c1d6737b6ff86a propertyValue 1 alignment 0 2 Microsoft.PowerShell.Commands.Internal.Format.FormatPropertyField ClassId2e4f51ef21dd47e99d3c952918aff9cd 78b102e894f742aca8c1d6737b6ff86a propertyValue 8.000 alignment 0 multiLine False outOfBand False writeStream None

ClassId2e4f51ef21dd47e99d3c952918aff9cd,4ec4f0187cb04f4cb6973460dfe252df
groupingEntry,<null>


ClassId2e4f51ef21dd47e99d3c952918aff9cd,cf522b78d86c486691226b40aa69e95c
groupingEntry,<null>


---
## Part 2: Extract (The Annoying ETL Part)

Now the classic ETL challenge: extract data from the PDF and insert into normalized tables.

First pass: **Extract exactly as-is.** Bad data and all.

In [12]:
$extractPrompt = "Extract the data from this pet vaccination PDF.

I have vaccine tables in tempdb on localhost (sql server/windows auth),
figure out the structure, insert the data from the PDF then show me what
you inserted"

$insertScript = Get-ChildItem ./Tests/pdf/immunization.pdf |
    ConvertTo-AITImage |
    Invoke-AITool -Prompt $extractPrompt -Model Haiku

In [ ]:
# Let's see what we imported in SSMS

### Notice anything wrong?

The data is in there exactly as the PDF showed it. But there might be problems...

---
## Part 3: Upsert with AI Data Quality Review

Now the magic. Ask Claude to review the data, identify problems, fix them, AND document what it found.

In [ ]:
Get-ChildItem ./Tests/pdf/immunization.pdf |
    ConvertTo-AITImage |
    Invoke-AITool -Prompt C:\github\aitools\unattended.md

Now let me read the PDF image more carefully to extract all the details.

Based on my analysis of the PDF document and the database data, here are my findings:

## Data Quality Issues Discovered

### 1. **BREED MISMATCH (CRITICAL)**
- **PDF Image**: Shows a small, fluffy gray/white dog that appears to be a **Schnauzer** or similar terrier breed
- **PDF Text & Database**: Lists breed as "Labrador"
- **Issue**: Labradors are large, short-haired dogs (55-80 lbs). The dog in the photo is clearly a small, long-haired breed

### 2. **Vaccination Date Before Birth Date**
- **Date of Birth**: February 25, 2014
- **Distemper vaccination**: February 1, 2014 (24 days BEFORE the dog was born!)
- **Parainfluenza vaccination**: February 1, 2014 (also before birth)

### 3. **Missing Vaccination Records**
The PDF shows a "Lyme Disease" third vaccination date but the database only has 2 Lyme Disease records for this pet (RecordId 16 and 17)

### 4. **State Abbreviation Issue**
- **PDF shows**: Mount Pl

Error: Cannot find drive. A drive with the name 'Review this pet vaccination PDF carefully for data quality issues.

I've already imported the data exactly as shown in the PDF into tempdb tables
(look it up on localhost).
Now I need you to' does not exist.

---
## Part 4: Data Quality Report

Generate a markdown report documenting all the issues found. 

This is your audit trail - proof that AI caught problems human review might miss.

In [ ]:
$reportPrompt = @"
Review this pet vaccination PDF and generate a data quality report.

Analyze the document thoroughly for:
- Data inconsistencies (breed vs. physical characteristics, weight ranges, etc.)
- Missing or incomplete information
- Suspicious or potentially incorrect values
- Format issues
- Any other data quality concerns

Output a well-formatted Markdown report with:
- Executive summary
- Table of issues found (columns: Field, Value in PDF, Issue, Recommended Fix)
- Confidence level for each finding
- Recommendations for human review

Be thorough - this report helps humans catch shit data before it pollutes the database.
"@

$report = Get-ChildItem ./Tests/pdf/immunization.pdf |
    ConvertTo-AITImage |
    Invoke-AITool -Prompt $reportPrompt

# Save the report
$report | Set-Content ./Tests/pdf/data_quality_report.md
Write-Host "Report saved to: ./Tests/pdf/data_quality_report.md" -ForegroundColor Green

# Display it
$report

---
## The Point

### What I Used to Teach
```powershell
# 1. Define rigid JSON schema (50 lines)
# 2. Configure structured output API
# 3. Write validation rules (100+ lines)
# 4. Build table mappings
# 5. Handle edge cases
# 6. Debug parsing failures
# 7. Hope you thought of everything
```

### What I Do Now
```powershell
"Review this PDF, create normalized tables, import the data, 
 find the problems, fix them, and document what you found."
```

---

### Why This Works

| Traditional ETL | AI-Assisted ETL |
|-----------------|----------------|
| Check against rules I thought of | Notices things I didn't think to check |
| Rigid schema matching | Semantic understanding |
| Fails on edge cases | Adapts to messy data |
| Silent data corruption | Documents concerns for review |

**The key insight:** AI catches semantic issues that rule-based validation can't.

"Labrador" is a valid breed. Traditional validation would pass it.

But AI looks at the *whole picture* and says "wait, this doesn't add up."